In [133]:
import numpy as np
import quant_rotor.core.hamiltonian as h
import quant_rotor.core.hamiltonian_big as hb
from itertools import product

# t_1 amplitutdes. (work)

In [302]:
sites_old = 3
states_old = 11

g = 0.1

H_K_V = h.hamiltonian(states_old, sites_old, g, clean=True)

sites = 3
states = 11

H_5 =  h.hamiltonian(states, sites, g, H_K_V)

eig_val, eig_vec = np.linalg.eig(H_5[0])

In [317]:
num_sites=5
num_states=5

In [318]:
def generate_quantum_rotor_configurations(num_sites: int, num_states: int) -> np.ndarray:
    """
    Generate all possible configurations for a quantum rotor system.
    
    Args:
        num_sites (int): Number of rotor sites.
        num_states (int): Number of discrete states per rotor.
                          Typically, this is 2*m_max + 1 for m in [-m_max, ..., m_max].
    
    Returns:
        np.ndarray: Array of shape (num_states ** num_sites, num_sites) containing all configurations.
    """
    return np.array(list(product(range(num_states), repeat=num_sites)))

# Example usage:
configs = generate_quantum_rotor_configurations(num_sites, num_states)
# print(configs)

In [319]:
for site in range(num_sites):
    for state in range(1,num_states):

        print("site:", site, "state:", state)

        index = state*num_states**site
        print("state", configs[index])

site: 0 state: 1
state [0 0 0 0 1]
site: 0 state: 2
state [0 0 0 0 2]
site: 0 state: 3
state [0 0 0 0 3]
site: 0 state: 4
state [0 0 0 0 4]
site: 1 state: 1
state [0 0 0 1 0]
site: 1 state: 2
state [0 0 0 2 0]
site: 1 state: 3
state [0 0 0 3 0]
site: 1 state: 4
state [0 0 0 4 0]
site: 2 state: 1
state [0 0 1 0 0]
site: 2 state: 2
state [0 0 2 0 0]
site: 2 state: 3
state [0 0 3 0 0]
site: 2 state: 4
state [0 0 4 0 0]
site: 3 state: 1
state [0 1 0 0 0]
site: 3 state: 2
state [0 2 0 0 0]
site: 3 state: 3
state [0 3 0 0 0]
site: 3 state: 4
state [0 4 0 0 0]
site: 4 state: 1
state [1 0 0 0 0]
site: 4 state: 2
state [2 0 0 0 0]
site: 4 state: 3
state [3 0 0 0 0]
site: 4 state: 4
state [4 0 0 0 0]


In [320]:
for i in range(num_sites):
    for j in range(1,num_states):
        for x in range(num_sites):
            for y in range(1,num_states):

                if i < x:
                    index = j*num_states**i + y*num_states**x
                    print("state", configs[index])

state [0 0 0 1 1]
state [0 0 0 2 1]
state [0 0 0 3 1]
state [0 0 0 4 1]
state [0 0 1 0 1]
state [0 0 2 0 1]
state [0 0 3 0 1]
state [0 0 4 0 1]
state [0 1 0 0 1]
state [0 2 0 0 1]
state [0 3 0 0 1]
state [0 4 0 0 1]
state [1 0 0 0 1]
state [2 0 0 0 1]
state [3 0 0 0 1]
state [4 0 0 0 1]
state [0 0 0 1 2]
state [0 0 0 2 2]
state [0 0 0 3 2]
state [0 0 0 4 2]
state [0 0 1 0 2]
state [0 0 2 0 2]
state [0 0 3 0 2]
state [0 0 4 0 2]
state [0 1 0 0 2]
state [0 2 0 0 2]
state [0 3 0 0 2]
state [0 4 0 0 2]
state [1 0 0 0 2]
state [2 0 0 0 2]
state [3 0 0 0 2]
state [4 0 0 0 2]
state [0 0 0 1 3]
state [0 0 0 2 3]
state [0 0 0 3 3]
state [0 0 0 4 3]
state [0 0 1 0 3]
state [0 0 2 0 3]
state [0 0 3 0 3]
state [0 0 4 0 3]
state [0 1 0 0 3]
state [0 2 0 0 3]
state [0 3 0 0 3]
state [0 4 0 0 3]
state [1 0 0 0 3]
state [2 0 0 0 3]
state [3 0 0 0 3]
state [4 0 0 0 3]
state [0 0 0 1 4]
state [0 0 0 2 4]
state [0 0 0 3 4]
state [0 0 0 4 4]
state [0 0 1 0 4]
state [0 0 2 0 4]
state [0 0 3 0 4]
state [0 0

In [275]:
def intermediate_normalisation(eig_val: np.ndarray, eig_vec: np.ndarray)->np.ndarray:

    min_index = np.argmin(eig_val)

    reference_ground_state = eig_vec[:, min_index]

    d = reference_ground_state / reference_ground_state[0]

    return d

def t_1_amplitutde(site_a: int, state_a: int, states: int, d: np.ndarray)-> int:
    index = state_a*states**site_a
    return d[index]

def t_2_amplitutde(site_a: int, state_a: int, site_b: int, state_b: int, states: int, d: np.ndarray)-> int:
    t_1_a = t_1_amplitutde(site_a, state_a, states, d)
    t_1_b = t_1_amplitutde(site_b, state_b, states, d)

    C_ab = d[state_a*states**site_a + state_b*states**site_b]

    t_2_ab = C_ab - t_1_a*t_1_b

    return t_2_ab, C_ab

In [304]:
d = intermediate_normalisation(eig_val, eig_vec)
site_a = 0
state_a = 1
site_b = 1
state_b = 1
t_1_a = t_1_amplitutde(site_b, state_b, states, d)
t_2_ab = t_2_amplitutde(site_a, state_a, site_b, state_b, states, d)
print(t_1_a)
print(t_2_ab[0])

(-5.425056681426581e-18+0j)
(-0.0394014771952756+0j)


In [277]:
site_c = 2
state_c = 1
site_d = 0
state_d = 1
t_1_c = t_1_amplitutde(site_c, state_c, states, d)
t_2_cd = t_2_amplitutde(site_c, state_c, site_d, state_d, states, d)

print(t_1_c)
print(t_2_cd[0])

(-1.6540065711986537e-16+0j)
(-0.03926454595099672+0j)


# t_amplitudes. (final)

In [ ]:
def intermediate_normalisation(eig_val: np.ndarray, eig_vec: np.ndarray)->np.ndarray:

    min_index = np.argmin(eig_val)

    reference_ground_state = eig_vec[:, min_index]

    d = reference_ground_state / reference_ground_state[0]

    return d

def t_1_amplitutde(site_a: int, state_a: int, states: int, d: np.ndarray)-> int:
    index = state_a*states**site_a
    return d[index]

def t_2_amplitutde(site_a: int, state_a: int, site_b: int, state_b: int, states: int, d: np.ndarray)-> int:
    t_1_a = t_1_amplitutde(site_a, state_a, states, d)
    t_1_b = t_1_amplitutde(site_b, state_b, states, d)

    C_ab = d[state_a*states**site_a + state_b*states**site_b]

    t_2_ab = C_ab - t_1_a*t_1_b

    return t_2_ab, C_ab

# t_amplitudes energy. (work)

In [327]:
def amplitute_energy(sites: int, states: int, g: float, d: np.ndarray):

    K, V = h.write_matrix_elements((states-1) // 2)

    V = V + V.T - np.diag(np.diag(V))
    V = V.reshape(states,states,states,states)
    V *= g 

    K = h.basis_m_to_p_matrix_conversion(K)
    V = h.basis_m_to_p_matrix_conversion(V)

    E_0 = K[0,0]*sites + np.einsum("ijij->", V) * states**sites

    sum_t_1 = 0
    sum_t_2 = 0

    for site_a in range(sites):
        for state_a in range(1, states):

            t1 = t_1_amplitutde(site_a, state_a, states, d)
            sum_t_1 += K[state_a, 0] * t1

            for site_b in range(site_a + 1, sites):
                for state_b in range(1, states):
                    
                    C2 = t_2_amplitutde(site_a, state_a, site_b, state_b, states, d)[1]
                    sum_t_2 += V[state_a, state_b, 0, 0] * C2

    print(E_0)
    print(sum_t_1)
    print(sum_t_2)

    return E_0 + sum_t_1 + sum_t_2
        

In [328]:
amplitute_energy(sites, states, g, d)

0.0
0j
(-0.020101729050886804+0j)


(-0.020101729050886804+0j)

In [326]:
min_index = np.argmin(eig_val)
eig_val[min_index]

(-0.02010172905076164+0j)

# t_amplitudes energy. (final)

In [331]:
def amplitute_energy(sites: int, states: int, g: float, d: np.ndarray):

    K, V = h.write_matrix_elements((states-1) // 2)

    V = V + V.T - np.diag(np.diag(V))
    V = V.reshape(states,states,states,states)
    V *= g 

    K = h.basis_m_to_p_matrix_conversion(K)
    V = h.basis_m_to_p_matrix_conversion(V)

    E_0 = K[0,0]*sites + np.einsum("ijij->", V) * states**sites

    sum_t_1 = 0
    sum_t_2 = 0

    for site_a in range(sites):
        for state_a in range(1, states):

            t1 = t_1_amplitutde(site_a, state_a, states, d)
            sum_t_1 += K[state_a, 0] * t1

            for site_b in range(site_a + 1, sites):
                for state_b in range(1, states):
                    
                    C2 = t_2_amplitutde(site_a, state_a, site_b, state_b, states, d)[1]
                    sum_t_2 += V[state_a, state_b, 0, 0] * C2

    print(E_0)
    print(sum_t_1)
    print(sum_t_2)

    return E_0 + sum_t_1 + sum_t_2
        